# Weather Data

Data is sourced from NOAA, through their free [Climate Data Online](https://www.ncdc.noaa.gov/cdo-web/) service.

In [4]:
import csv

with open('data.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
  
header = data[0]
data = data[1:]

print(header)

['STATION', 'NAME', 'DATE', 'AWND', 'PGTM', 'PRCP', 'SNOW', 'SNWD', 'TAVG', 'TMAX', 'TMIN', 'WDF2', 'WDF5', 'WSF2', 'WSF5', 'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 'WT09']


This includes data from the following stations:

| Station ID | City |
| --- | --- |
| USW00023174 | Los Angeles |
| USW00026451 | Anchorage |
| USW00014739 | Boston |

Each station reports the following data:

* Average Wind
* Precipitation
* Average Temperature

In [5]:
from dataclasses import dataclass

@dataclass
class Report:
    station: str
    date: str
    wind: float
    precip: float
    temp: float

In [6]:
reports = {}

for row in data:
    report = Report(
        row[header.index("STATION")],
        row[header.index("DATE")],
        float(row[header.index("AWND")]), # average wind
        float(row[header.index("PRCP")]), # precipitation
        float(row[header.index("TMAX")]) # max temp
    )
    if report.station not in reports:
        reports[report.station] = []
    reports[report.station].append(report)

print(*[f"{station}: {len(reports[station])} reports" for station in reports], sep='\n')

USW00023174: 365 reports
USW00026451: 365 reports
USW00014739: 365 reports


Audio is generated as MIDI files by traversing chronologically through the readings.

In [21]:
import os
import mido

os.makedirs("output", exist_ok=True)
file = mido.MidiFile()

for station, reports in reports.items():
    track = mido.MidiTrack()

    file.tracks.append(track)

    track.append(mido.Message('program_change', program=0, time=0))
    track.append(mido.MetaMessage('track_name', name=station, time=0))

    for report in reports:
        track.append(mido.Message('note_on', note=int(round(report.temp)), velocity=64, time=0))
        track.append(mido.Message('control_change', control=1, value=int(round(report.wind * 3)), time=0))
        track.append(mido.Message('note_off', note=int(round(report.temp)), velocity=64, time=20))

file.save(f'output/weather.mid')